# Question 1

In [1]:
import pandas as pd
import time
import string
import time

Loading the password files

In [2]:
pass1 = open("passwords1.txt").read().splitlines()
pass2 = open("passwords2.txt").read().splitlines()

Defining the 2 hash functions to be used:

Method: For each existing string character i assign a value going from 0 to n (number of characters). Then i perform some transformations to make the hashes random. I finally mod with a prime number to minimise collisions.


In [3]:
def myhash1(s):
   
    a = string.printable
    lst = []
    for i in a:
        lst.append(i)  
        
    encoder = []
    for j in range(len(lst)):
        encoder.append(int((((j+100)**2)*5059)+47))
            

    keys= lst
    values= encoder
    dictionary = dict(zip(keys, values))
   
    empty= []
    for e in s:
        empty.append(dictionary.get(e))
    str1 =  int(''.join(map(str, empty)))
    return str1%861259057

def myhash2(s):
    
    a = string.printable
    lst = []
    for i in a:
        lst.append(i)  
        
    encoder = []
    for j in range(len(lst)):
        encoder.append(int(10 + ((j+100)**(3/2))*67))
            

    keys= lst
    values= encoder
    dictionary = dict(zip(keys, values))
   
    empty= []
    for e in s:
        empty.append(dictionary.get(e))
    str1 =  int(''.join(map(str, empty)))
    return str1%853969703

# Hashing passwords. 

Running time note: I believe the hashing of the passwords should not be included in the bloom filter function since most passwords are stored in their hash equivalent and also hashing the whole database 1 time over actually takes a very long time.
The running time of my hash functions is between 0.0006 and 0.0008 per string. Total strings to hash are more than 100 million, with 2 hashes this takes about 6 to 8 hours. Hence the actual bloom filter will be used on the list of already hashed passwords. Since the hash function was changed the new hashlists are based on a cut dataset to speed up running time without losing the logic of the filter. A prediction on running time for the whole dataset will be made at the end.

In [4]:
pass1 = pass1[:1000000]
pass2 = pass2[:390000]

Hashing existing passwords: passwords1

In [5]:
hash1p1 = [myhash1(str(pass1[i])) for i in range(len(pass1))]
hash2p1 = [myhash2(str(pass1[i])) for i in range(len(pass1))]

Hashing all the possible new passwords: passwords2

In [6]:
hash1p2 = [myhash1(str(pass2[i])) for i in range(len(pass2))]
hash2p2 = [myhash2(str(pass2[i])) for i in range(len(pass2))]

The hashes look like this:

In [9]:
print(hash1p1[:10])

[368678279, 188403858, 707803710, 105061637, 269377512, 423937066, 6527838, 673589225, 12503355, 278690290]


# Bloom Filter

Creating the bloom filter which is initially a containter list of all 0s. Length based on range of all possible hashes, determined by largest prime used in modding the hashes string.

In [10]:
def BloomFilter():
    start = time.time()
    bf0 = [0]*861259057

#Joining the lists(set union) of the 2 hash functions applied to the existing passwords(passwords1)
    hashlist = [hash1p1,hash2p1]
    listunion =  list(set().union(*hashlist))
    L = len(listunion)


#Filling in the bloom filter with 1 for every index that was created from hash functions
    for i in range(L):
        bf0[listunion[i]] = 1

#Now we have to see if the possible new passwords already exist according to out bloom filter.
    
    newhashlist = [hash1p2,hash2p2]

#Now I creat a yesno list saying if the potential new passwords exist or not, if both has indexes in the bloom filter equal 1, then exists, else they don't.

    yesno= []
    for i in range(len(hash1p2)):
        if bf0[newhashlist[0][i]] == 1 and bf0[newhashlist[1][i] == 1]:
            yesno.append(1)
        else:
            yesno.append(0)
    end = time.time()
    
#To calculate the probability of false positives i reference this source: https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=903775
  #and use Bloom's classical formulae for the false positive rate.

    m = 861259057#Length of bloom filter
    k = 2 #Number of hash functions used
    n = 1000000 #objects mapped in bloom filter(passwords1)
    p = (1-((1-1/m)**(k*n)))**k
  
    print('Number of hash function used: ', 2)
    print('Number of duplicates detected: ', sum(yesno))
    print('Probability of false positives: ', p)
    print('Execution time: ', end-start)


Bloom Filter steps:

1 - Initialising filter
2 - Joining the lists(set union) of the 2 hash functions applied to the existing passwords(passwords1)
3 - Filling the bloom filter with each index found by both hash functions
4 - Creating another list containing the prediction of our bloom filter
5 - Filling this list with 1s if both hashes of possible new passwords exist in bloom filter, 0 otherwise. 1 in "yesno" list means password from password2 database probably exists in password1 database already, 0 means password for sure does not exist.


Probability of false positives (FPR)
To calculate the probability of false positives i reference this source: https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=903775
I use Bloom's classical formulae for FPR. This underestimates the actual FPR of my bloom filter as proved in the paper but more accurate formulaes provided in the paper are computationally inefficient if not impossible with large datasets.

# BloomFilter Output/Final Comments

In [12]:
BloomFilter()

Number of hash function used:  2
Number of duplicates detected:  0
Probability of false positives:  5.380022741846551e-06
Execution time:  6.916478872299194


No duplicates were detected which is actually to be expected given the password datasets are made up of randomized 20 character strings.
Execution time on whole dataset which is 100 times larger is expected to be about 100 times larger since BloomFilter() is O(n). Therefore execution time would be around 400-700 seconds (guess of the average execition times)/ between 6 and 12 minutes on this particular machine. This particular bloom filter has a really low theoretical probability of false positives. Therefore the size of the filter could be reduced to lower execution time because the probability of false positives would still be low.

# Bonus

The exact number of false positives I have found is 0 since i detected 0 duplicates. If I did detect a duplicate I would find the index of the duplicate detected from hashing, find its corresponding string representation (the actual password from passwords2.txt file) and then I would run a search through the first database "passwords1.txt" and see if there is actually a match. If there is not then I would count that as a false positive.